In [1]:
import pandas as pd
import config

In [9]:
columns = pd.read_csv('usa_spending_data.csv', nrows=0).columns.to_list()

In [11]:
matching_description = [col for col in columns if 'description' in col]

In [ ]:
with open('columns.txt', 'w') as f:
    for col in columns:
        f.write(col)
        f.write('\n')

In [31]:
# 1. Okunacak kolonlar (CSV'den çekilecekler)
critical_columns = [
    'contract_award_unique_key', 'award_id_piid', 'recipient_uei',
    'federal_action_obligation', 'total_dollars_obligated', 
    'total_outlayed_amount_for_overall_award',
    'current_total_value_of_award', # <--- BURADA EKSİK OLABİLİR, KONTROL ET
    'base_and_all_options_value',
    'action_date', 'recipient_name', 'transaction_description'
]

# 2. Sayıya çevrilecek kolonlar (Mutlaka yukarıdaki listede de olmalı!)
financial_columns = [
    'federal_action_obligation',
    'total_dollars_obligated',
    'total_outlayed_amount_for_overall_award',
    'current_total_value_of_award',
    'base_and_all_options_value'
]

In [3]:
db = config.db_config

In [ ]:
from sqlalchemy import create_engine
db_url = f'postgresql://{db['username']}:{db['password']}@{db['host']}:{db['port']}/{db['database']}'

engine = create_engine(db_url)

In [53]:
recipient_columns = ['recipient_uei', 'recipient_name']
award_columns = ['contract_award_unique_key', 'award_id_piid', 'transaction_description']

recipients_df = pd.read_csv('usa_spending_data.csv', usecols=recipient_columns)
award_df = pd.read_csv('usa_spending_data.csv', usecols=award_columns)

In [ ]:
recipients_df = recipients_df.drop_duplicates(subset=['recipient_uei']).astype('string').apply(lambda x: x.str.strip()).fillna('unknown')
award_df = award_df.drop_duplicates(subset=['contract_award_unique_key']).astype('string').apply(lambda x: x.str.strip()).fillna('unknown')

recipients_df.to_sql('dim_recipient', con=engine, index=False, method='multi', if_exists='append')
award_df.to_sql('dim_award', con=engine, index=False, method='multi', if_exists='append')

In [60]:
recipients_df.to_sql('dim_recipient', con=engine, index=False, method='multi', if_exists='append')
award_df.to_sql('dim_award', con=engine, index=False, method='multi', if_exists='append')

9938

In [73]:
with engine.connect() as conn:
    df_award = pd.read_sql("select award_key, contract_award_unique_key from dim_award", conn)

dict_award = dict(zip(df_award.contract_award_unique_key, df_award.award_key))

with engine.connect() as conn:
    df_recipient = pd.read_sql('select recipient_key, recipient_uei from dim_recipient', conn)

dict_recipient = dict(zip(df_recipient.recipient_uei, df_recipient.recipient_key))

In [74]:
csv_columns = [
    'contract_award_unique_key', 'recipient_uei', 'action_date',
    'federal_action_obligation', 'total_dollars_obligated', 
    'total_outlayed_amount_for_overall_award', 
    'current_total_value_of_award', 
    'base_and_all_options_value'
]

In [75]:
fact_df = pd.read_csv('usa_spending_data.csv', usecols=csv_columns)

fact_df['contract_award_unique_key'] = fact_df['contract_award_unique_key'].map(dict_award)
fact_df['recipient_uei'] = fact_df['recipient_uei'].map(dict_recipient)

fact_df = fact_df.rename(columns={
    'contract_award_unique_key': 'award_key', 
    'recipient_uei': 'recipient_key',
    'total_outlayed_amount_for_overall_award': 'total_outlayed_amount',
    'current_total_value_of_award': 'current_total_value',
    'base_and_all_options_value': 'base_and_options_value'
})

fact_df = fact_df.dropna(subset=['award_key', 'recipient_key'])

In [79]:
fact_df['action_date'] = pd.to_datetime(fact_df['action_date'])
fact_df['total_outlayed_amount'] = fact_df['total_outlayed_amount'].fillna(0)
fact_df['current_total_value'] = fact_df['current_total_value'].fillna(0)
fact_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23130 entries, 0 to 23129
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   award_key                  23130 non-null  int64         
 1   federal_action_obligation  23130 non-null  float64       
 2   total_dollars_obligated    23130 non-null  float64       
 3   total_outlayed_amount      23130 non-null  float64       
 4   current_total_value        23130 non-null  float64       
 5   base_and_options_value     23130 non-null  float64       
 6   action_date                23130 non-null  datetime64[ns]
 7   recipient_key              23130 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 1.4 MB


In [80]:
fact_df.to_sql('fact_spending', con=engine, index=False, method='multi', if_exists='append')

23130